# Text classification

## Step 1 import relevant packages

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

## Step 2 Load data

In [2]:
import pandas as pd

data = pd.read_csv("https://raw.githubusercontent.com/SophonPlus/ChineseNlpCorpus/master/datasets/ChnSentiCorp_htl_all/ChnSentiCorp_htl_all.csv")
data = data.dropna()
data

label                                             review
0         1  距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较...
1         1                       商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2         1         早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3         1  宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4         1               CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风
...     ...                                                ...
7761      0  尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762      0  盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763      0  看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764      0  我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...
7765      0  说实在的我很失望，之前看了其他人的点评后觉得还可以才去的，结果让我们大跌眼镜。我想这家酒店以...

[7765 rows x 2 columns]

## Step 3 Create a dataset

In [3]:
from torch.utils.data import Dataset

In [4]:
class MyDataset(Dataset):
    def __init__(self):
        self.data = pd.read_csv("https://raw.githubusercontent.com/SophonPlus/ChineseNlpCorpus/master/datasets/ChnSentiCorp_htl_all/ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data.iloc[idx]['review'], self.data.iloc[idx]['label']


In [5]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', np.int64(1))
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', np.int64(1))
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', np.int64(1))
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', np.int64(1))
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', np.int64(1))


## Step 4 Split the dataset

In [6]:
from torch.utils.data import random_split

In [7]:
trainset, validset = random_split(dataset, [0.9, 0.1])
len(trainset), len(validset)

(6989, 776)

In [8]:
for i in range(10):
    print(trainset[i])

('酒店太旧了，大堂感觉象三星级的，房间也就是的好点的三星级的条件，在青岛这样的酒店是绝对算不上四星标准，早餐走了两圈也没有找到可以吃的，太差了', np.int64(0))
('酒店有些老，设施一般，早餐一般。位置位于火车站附近，但是周边环境有些乱。', np.int64(1))
('因为客户就在旁边一栋楼，所以就近住了，不过这个地方好象也就只要这么一个大点的宾馆吧，同事说好象是日本人开的', np.int64(0))
('酒店总体可以,出门交通不太方便，离市中心较远，房内设施有点旧,电视机太小且旧,配不上五星级。饮食酒店配套还可以，中式，西式，日式都有。要提的一点就是房间没有提供收费的卫生用品（如一次性毛巾，安全套，清洁洗液等），我打电话要安全套，酒店房间清洁阿姨跟一同童装束的人抢着送来，一问之下每个收费50元，也太贵了，难怪他们抢着送来，可能他们在赚外快，不知道酒店对这方面有没有管理。', np.int64(1))
('可能是建的时间比较早吧,感觉内部不是很突出了.', np.int64(1))
('大堂有点小，房间还不错，定的是商务套，客厅很大，还有小厨房可以根据需求使用，卧室很小。饭店出去，沿右手一条小区里的小路一直走到路边左拐，有个茶餐厅，还不错。', np.int64(1))
('酒店的位置很好，走出去就是松花江畔，如果住靠江边的房间，早点起来推开窗就有机会看到雾凇了。交通也比较方便，从火车站打车过去也就是跳一次表。酒店的设施半新不旧，感觉是老三星级宾馆翻新过的。房间比较大，但是布局不怎么好。淋浴头不够大，总是水量不足的样子。', np.int64(1))
('这个酒店的所谓河畔全景房就是靠近拉萨河这边的房间,网友说的比较吵是因为在房子和河流之间有一条马路,整天都有车辆经过,比较吵就不奇怪了!', np.int64(1))
('房间的格局在风水中属败相,卫生间马桶堵塞,无奈的一晚!!!', np.int64(0))
('房间不错,但服务质素极低,服务员态度生硬,冷淡.入住两晚,虽然大部份时间都在外游玩,但也没少看服务员白眼,好象咨询下的士的状况也打扰了该酒店高贵的服务员.', np.int64(0))


## Step 5 Create a Dataloader

In [9]:
import torch

In [10]:
tokenizer = AutoTokenizer.from_pretrained('hfl/rbt3')

def collate_fn(batch):
    texts, labels = zip(*batch)

    inputs = tokenizer(list(texts), max_length=128, padding='max_length', truncation=True, return_tensors='pt')
    inputs['labels'] = torch.tensor(labels)
    return inputs

In [11]:
from torch.utils.data import DataLoader

trainloader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_fn)
validloader = DataLoader(validset, batch_size=64, shuffle=False, collate_fn=collate_fn)

## Step 6 Create a model and an optimizer

In [12]:
from torch.optim import Adam

In [13]:
model = AutoModelForSequenceClassification.from_pretrained('hfl/rbt3', num_labels=2)
device = torch.device('mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(f"Using device: {device}")
optimizer = Adam(model.parameters(), lr=2e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using device: mps


## Step 7 Train and validate

In [14]:
def evalutate():
    model.eval()
    with torch.inference_mode():
        for batch in validloader:
            batch = {k: v.to(device) for k, v in batch.items()}  # Move batch to device
            output = model(**batch)
            logits = output.logits
            predictions = logits.argmax(dim=-1)
            labels = batch['labels']
            acc = (predictions == labels).float().mean().item()
    return acc

def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            batch = {k: v.to(device) for k, v in batch.items()}  # Move batch to device
            optimizer.zero_grad()
            output = model(**batch)
            loss = output.loss
            loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f"Epoch {ep}, Step {global_step}, Loss: {loss.item()}")
            global_step += 1
        acc = evalutate()
        print(f"Epoch {ep} Validation Accuracy: {acc:.4f}")


## Step 8 Train the model

In [15]:
train()

Epoch 0, Step 0, Loss: 0.9294122457504272
Epoch 0, Step 100, Loss: 0.348019003868103
Epoch 0, Step 200, Loss: 0.28286707401275635
Epoch 0 Validation Accuracy: 0.8750
Epoch 1, Step 300, Loss: 0.2515795826911926
Epoch 1, Step 400, Loss: 0.3003975749015808
Epoch 1 Validation Accuracy: 0.8750
Epoch 2, Step 500, Loss: 0.14660191535949707
Epoch 2, Step 600, Loss: 0.17113257944583893
Epoch 2 Validation Accuracy: 0.8750


## Step 9 Prediction

In [16]:
sen = "这部电影真是太棒了！"
id2_label = {0: "差评！", 1: "好评！"}
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors='pt').to(device)
    output = model(**inputs)
    logits = output.logits
    prediction = logits.argmax(dim=-1).item()
    print(f"Prediction for '{sen}': {id2_label[prediction]}")
    print(f"Logits: {logits}")

Prediction for '这部电影真是太棒了！': 好评！
Logits: tensor([[-1.4091,  1.2562]], device='mps:0')


In [17]:
from transformers import pipeline

In [18]:
model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

Device set to use mps:0


In [19]:
pipe(sen)

[{'label': '好评！', 'score': 0.9349479079246521}]